In [15]:
import scipy.stats
import numpy as np            
import csv
import ast

In [16]:
def entropy(class_y):
    # Input:            
    #   class_y         : list of class labels (0's and 1's)
    
    # TODO: Compute the entropy for a list of classes
    #
    # Example:
    #    entropy([0,0,0,1,1,1,1,1,1]) = 0.92
    n_y = len(class_y)

    if n_y <= 1:
        return 0

    value,counts = np.unique(class_y, return_counts=True)
    probs = counts / n_y
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    return -np.sum(probs * np.log2(probs))

In [17]:
def partition_classes(X, y, split_attribute, split_val):
    nx = np.array(X)
    ny = np.array(y)
    
    if type(split_val) == str:
        colx = nx[:,split_attribute].astype(str)
        idx = colx == split_val
        
    else:
        colx = nx[:,split_attribute].astype(float)
        idx = colx <= split_val

    X_left = nx[idx].tolist()
    X_right = nx[~idx].tolist()
    y_left = ny[idx].tolist()
    y_right = ny[~idx].tolist()
    
    return (X_left, X_right, y_left, y_right)

In [18]:
def information_gain(previous_y, current_y):
    info_gain = 0
    info_gain = entropy(previous_y)

    ylen = len(previous_y)
    for y in current_y:
        info_gain -= float(len(y) * entropy(y))/ylen

    return info_gain

In [70]:
class DecisionTree(object):
    def __init__(self):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = {}
        self.max_depth = 5

    def learn(self, X, y):
        # TODO: Train the decision tree (self.tree) using the the sample X and labels y
        # You will have to make use of the functions in utils.py to train the tree

        # One possible way of implementing the tree:
        #    Each node in self.tree could be in the form of a dictionary:
        #       https://docs.python.org/2/library/stdtypes.html#mapping-types-dict
        #    For example, a non-leaf node with two children can have a 'left' key and  a 
        #    'right' key. You can add more keys which might help in classification
        #    (eg. split attribute and split value)
        self.tree = self.build_tree(X, y, 1)

    def build_tree(self, X, y, depth):
        nx = np.array(X)
        (max_row, max_col) = nx.shape
        tree = {}
        print(depth, self.max_depth)
        
        if depth <= self.max_depth:
            # or len(counts) == 1:
            (vals, counts) = np.unique(y, return_counts=True) 
            idx = np.argmax(counts)
            if depth == self.max_depth or len(counts) == 1:
                tree['label'] = 'leaf'
                tree['value'] = vals[idx]
                return tree

            max_gain = 0
            for i in range(max_col):
                # print("processing X[%s]" % i)
                uniq_x = np.quantile(nx[:, i], [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
                for item in uniq_x:
                    # print("partitioning classes X[%s]: %s" % (i, item))
                    (X_left, X_right, y_left, y_right) = partition_classes(X, y, i, item)
                    # print('calculating gain...')
                    gain = information_gain(y, [y_left, y_right])
                    if (gain > max_gain):
                        # print('found new max gain: %s' % gain)
                        max_gain = gain
                        attr_idx = i
                        attr_value = item
                        X1 = X_left
                        y1 = y_left
                        X2 = X_right
                        y2 = y_right
            
            tree['label'] = 'normal'
            tree['attr'] = attr_idx
            tree['split'] = attr_value
            left = self.build_tree(X1, y1, depth + 1)
            tree['left'] = left
            right = self.build_tree(X2, y2, depth + 1)
            tree['right'] = right
            return tree

        
    def classify(self, record):
        # TODO: classify the record using self.tree and return the predicted label
        node = self.tree
        while node['label'] == 'normal':
            if record[node['attr']] <= node['split']:
                node = node['left']
            else:
                node = node['right']
        if node['label'] == 'leaf':
            return node['value']

In [71]:
X = list()
y = list()
XX = list()  # Contains data features and data labels
numerical_cols = set([i for i in range(0, 9)])  # indices of numeric attributes (columns)

# Loading data set
print("reading pulsar_stars")
with open("pulsar_stars.csv") as f:
    next(f, None)
    for line in csv.reader(f, delimiter=","):
        xline = []
        for i in range(len(line)):
            if i in numerical_cols:
                xline.append(ast.literal_eval(line[i]))
            else:
                xline.append(line[i])

        X.append(xline[:-1])
        y.append(xline[-1])
        XX.append(xline[:])

reading pulsar_stars


In [72]:
tree = DecisionTree()
tree.learn(X, y)

1 5
2 5
3 5
4 5
5 5
5 5
4 5
5 5
5 5
3 5
4 5
5 5
5 5
4 5
5 5
5 5
2 5
3 5
4 5
5 5
5 5
4 5
5 5
5 5
3 5
4 5
5 5
5 5
4 5
5 5
5 5


In [62]:
tree.tree['left'].tree['left'].tree['left'].tree['left'].tree

AttributeError: 'dict' object has no attribute 'tree'

In [67]:
tree.tree

{'label': 'normal',
 'attr': 2,
 'split': 0.8918162523,
 'left': {'label': 'normal',
  'attr': 2,
  'split': 0.3337745231,
  'left': {'label': 'normal',
   'attr': 2,
   'split': 0.079124445,
   'left': {'label': 'normal',
    'attr': 2,
    'split': -0.09675876939999996,
    'left': {'label': 'leaf', 'value': 0},
    'right': {'label': 'leaf', 'value': 0}},
   'right': {'label': 'normal',
    'attr': 5,
    'split': 26.219759906,
    'left': {'label': 'leaf', 'value': 0},
    'right': {'label': 'leaf', 'value': 0}}},
  'right': {'label': 'normal',
   'attr': 5,
   'split': 32.542088650000004,
   'left': {'label': 'normal',
    'attr': 5,
    'split': 19.142482118,
    'left': {'label': 'leaf', 'value': 0},
    'right': {'label': 'leaf', 'value': 0}},
   'right': {'label': 'normal',
    'attr': 2,
    'split': 0.3963582204,
    'left': {'label': 'leaf', 'value': 0},
    'right': {'label': 'leaf', 'value': 0}}}},
 'right': {'label': 'normal',
  'attr': 2,
  'split': 1.6126868884999999,


In [77]:
tree.classify(X[19])

1

In [32]:
y[19]

1

In [76]:
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
